# 🔗 7.5 – Pandas: Manejo de Varias Tablas, Joins y Agregaciones

En este notebook aprenderás a **combinar y agrupar datos** provenientes de distintas tablas o fuentes.

Usaremos funciones clave como `merge()`, `concat()`, y `groupby()` para realizar **joins**, concatenaciones y **agregaciones estadísticas**.

In [35]:
import pandas as pd
from IPython.display import display

print("✅ Notebook 7.5 — Joins y Agregaciones cargado correctamente.")


✅ Notebook 7.5 — Joins y Agregaciones cargado correctamente.


---
## 1️⃣ Creación de DataFrames de ejemplo

Simularemos dos tablas: una de **productos** y otra de **ventas**.

In [36]:
productos = pd.DataFrame({
    'id_producto': [1, 2, 3, 4],
    'nombre': ['Monitor', 'Teclado', 'Ratón', 'Altavoces'],
    'precio': [120, 45, 25, 60]
})

ventas = pd.DataFrame({
    'id_venta': [101, 102, 103, 104, 105, 106],
    'id_producto': [1, 1, 2, 3, 3, 4],
    'cantidad': [2, 1, 3, 5, 2, 4],
    'region': ['Norte', 'Sur', 'Este', 'Oeste', 'Norte', 'Sur']
})

print('🧱 Productos:')
display(productos)

print('🧾 Ventas:')
display(ventas)

🧱 Productos:


,id_producto,nombre,precio
0,1,Monitor,120
1,2,Teclado,45
2,3,Ratón,25
3,4,Altavoces,60


🧾 Ventas:


,id_venta,id_producto,cantidad,region
0,101,1,2,Norte
1,102,1,1,Sur
2,103,2,3,Este
3,104,3,5,Oeste
4,105,3,2,Norte
5,106,4,4,Sur


✅ Tenemos una tabla de productos (referencial) y una tabla de ventas con identificadores comunes (`id_producto`).

---
## 2️⃣ Joins con `merge()`

`merge()` combina tablas usando una o varias claves comunes.

Por defecto realiza un **inner join** (solo filas coincidentes).

In [37]:
df_join = pd.merge(ventas, productos, on='id_producto', how='inner')
df_join.head()

,id_venta,id_producto,cantidad,region,nombre,precio
0,101,1,2,Norte,Monitor,120
1,102,1,1,Sur,Monitor,120
2,103,2,3,Este,Teclado,45
3,104,3,5,Oeste,Ratón,25
4,105,3,2,Norte,Ratón,25


✅ Podemos usar `how='left'`, `right`, o `outer` para incluir datos no coincidentes.

---
## 3️⃣ 🧩 Ejercicio 1 — Calcular ingresos totales

Usando el `DataFrame` combinado, crea una columna **`ingresos`** calculada como `precio × cantidad`.

Luego muestra el total de ingresos por región.

💡 *Pista:* usa `groupby('region')['ingresos'].sum()`.

In [38]:
# Tu código aquí...

### ✅ Solución propuesta

In [39]:
df_join['ingresos'] = df_join['precio'] * df_join['cantidad']
resumen_region = df_join.groupby('region', as_index=False)['ingresos'].sum().sort_values(by='ingresos', ascending=False)
resumen_region

,region,ingresos
3,Sur,360
1,Norte,290
0,Este,135
2,Oeste,125


---
## 4️⃣ Concatenación de DataFrames

Podemos apilar tablas vertical u horizontalmente con `pd.concat()`.

### Ejemplo: concatenar nuevas ventas

In [40]:
nuevas_ventas = pd.DataFrame({
    'id_venta': [107, 108],
    'id_producto': [2, 4],
    'cantidad': [2, 1],
    'region': ['Oeste', 'Norte']
})

ventas_total = pd.concat([ventas, nuevas_ventas], ignore_index=True)
ventas_total.tail()

,id_venta,id_producto,cantidad,region
3,104,3,5,Oeste
4,105,3,2,Norte
5,106,4,4,Sur
6,107,2,2,Oeste
7,108,4,1,Norte


✅ `ignore_index=True` reinicia el índice automáticamente.

---
## 5️⃣ Agregaciones con `groupby()`

El método `groupby()` agrupa filas según una o varias columnas y aplica funciones de agregación.

Ejemplo: obtener el **número de ventas y el promedio de cantidad por producto**.

In [41]:
resumen = df_join.groupby('nombre', as_index=False).agg(
    total_ventas=('cantidad', 'sum'),
    media_cantidad=('cantidad', 'mean'),
    total_ingresos=('ingresos', 'sum')
)
resumen

,nombre,total_ventas,media_cantidad,total_ingresos
0,Altavoces,4,4.0,240
1,Monitor,3,1.5,360
2,Ratón,7,3.5,175
3,Teclado,3,3.0,135


---
## 6️⃣ 🧩 Ejercicio 2 — Ranking de productos por ingresos

Usando el resumen anterior, agrega una columna **`ranking`** que ordene los productos del más al menos rentable.

💡 *Pista:* usa `.rank(method='dense', ascending=False)` sobre la columna de ingresos.

In [42]:
# Implementa tu código aquí...

### ✅ Solución propuesta

In [43]:
resumen['ranking'] = resumen['total_ingresos'].rank(method='dense', ascending=False).astype(int)
resumen.sort_values(by='ranking')

,nombre,total_ventas,media_cantidad,total_ingresos,ranking
1,Monitor,3,1.5,360,1
0,Altavoces,4,4.0,240,2
2,Ratón,7,3.5,175,3
3,Teclado,3,3.0,135,4


---
## 7️⃣ Funciones de agregación personalizadas

También podemos definir nuestras propias funciones y aplicarlas a los grupos.

In [44]:
def margen(grupo):
    precio_medio = grupo['precio'].mean()
    ingreso_real = (grupo['precio'] * grupo['cantidad']).sum()
    return ingreso_real - (precio_medio * grupo['cantidad'].sum() * 0.8)

margenes = df_join.groupby('nombre')[['precio', 'cantidad']].apply(margen)
margenes = margenes.round(2)
margenes.name = 'margen_estimado'
margenes


nombre
Altavoces    48.0
Monitor      72.0
Ratón        35.0
Teclado      27.0
Name: margen_estimado, dtype: float64

✅ `groupby().apply()` ejecuta una función sobre cada grupo y devuelve un resultado personalizado.

---
## 8️⃣ 🧠 Resumen del notebook

- `merge()` combina tablas por clave (joins: inner, left, right, outer).
- `concat()` apila tablas vertical u horizontalmente.
- `groupby()` agrupa y resume datos con funciones agregadas.
- `.agg()` y `.apply()` permiten cálculos personalizados.
- Estas técnicas son esenciales para análisis de negocio y preparación de datasets.

💡 Próximo paso → **7.6 – Laboratorio: Análisis de Ventas con Pandas y Numpy.**